In [ ]:
import pandas as pd
import os
import seaborn as sns
sns.set_theme()
import matplotlib.pyplot as plt

In [ ]:
# read the expression data from every file and combine them together
file_path = '/Users/yanlixu/Desktop/pingan/合作/CDK12/220616/LINSCP_for_drug_combinations'
files = os.listdir(file_path)

file_list = []
gene_list = []
diff_value_list = []

for file in files:
    if os.path.isfile(os.path.join(file_path, file)):
        file_list.append(file.replace(' .csv', ''))
        if file == 'LINCSCP_30171 .csv':
            gene_list = data['Name_GeneSymbol'].tolist()       
        data = pd.read_csv(os.path.join(file_path, file))
        diff_value_list.append(data['Value_LogDiffExp'].tolist())
        
diff_value_df = pd.DataFrame(diff_value_list)
diff_value_df = diff_value_df.transpose()

diff_value_df.columns = file_list
diff_value_df.index = gene_list
diff_value_df

In [ ]:
# read top up and down regulated genes data
top_genes_data = pd.read_excel('/Users/yanlixu/Desktop/pingan/合作/CDK12/220616/1_breast_cancer_20220616.xlsx', sheet_name='top_843_genes')

top_genes_data = top_genes_data.set_index('Name_GeneSymbol')
top_genes_data

In [ ]:
# merge two dataframe
df_merged = pd.merge(diff_value_df, top_genes_data[['Value_LogDiffExp']], left_index=True, right_index=True)
df_merged.rename(columns={'Value_LogDiffExp': 'breast_cancer_reverse_gene_expression'}, inplace=True)
df_merged

In [ ]:
#df_merged.to_csv('/Users/yanlixu/Desktop/pingan/合作/CDK12/220616/test0616_1.csv')

In [ ]:
ax = sns.clustermap(df_merged, standard_scale=1, cmap="vlag")
plt.show()

# method 1: covariance
# 以化合物XXX为例，这个化合物跟疾病reversed gene expression相关性越大，跟化合物XXX的基因表达相关性越小越好，综合考虑的话，前者除以后者的商越大，组合的后效果好的可能性越大。

In [ ]:
# calculate covariance
df_merged_cov = df_merged.cov()

In [ ]:
df_merged_cov

In [ ]:
ax = sns.clustermap(df_merged_cov, standard_scale=1, cmap="vlag")
plt.show()

In [ ]:
# sort the dataframe by the covariance with reversed gene expression of breast cancer
df_merged_cov_sorted = df_merged_cov.sort_values(by=['breast_cancer_reverse_gene_expression'], ascending=False)
# remove disease row
df_merged_cov_sorted_no_disease_row = df_merged_cov_sorted.drop(index='breast_cancer_reverse_gene_expression')
# split the dataframe
df_cov_1 = df_merged_cov_sorted_no_disease_row.drop('breast_cancer_reverse_gene_expression', 1)
df_cov_2 = df_merged_cov_sorted_no_disease_row['breast_cancer_reverse_gene_expression']
# transpose the dataframe
df_cov_1_transpose = df_cov_1.transpose()
# merge dataframe, every LINSCPXXX column and disease column combined show the distance of compound Y with LINSCPXXX and disease 
df_cov_final = pd.merge(df_cov_1_transpose, df_cov_2, left_index=True, right_index=True)
df_cov_final = df_cov_final.sort_values(by=['breast_cancer_reverse_gene_expression'], ascending=False)
df_cov_final

In [ ]:

        
col_names = df_cov_final.columns
for sample in col_names:
    if sample != "breast_cancer_reverse_gene_expression":  
        # rank the compounds
        df_cov_final[sample+'_rank_scove'] = round(df_cov_final['breast_cancer_reverse_gene_expression']/df_cov_final[sample], 2)
        plot = sns.scatterplot(data=df_cov_final, x=sample, y="breast_cancer_reverse_gene_expression")
        plot.set_xlabel(("covariance with " + sample), fontsize = 20)
        plot.set_ylabel("covariance with disease", fontsize = 20)
        plt.show()



    

In [ ]:
df_cov_final.to_csv('df_cov_final.csv')

# method2: correlation

# 以化合物XXX为例，这个化合物跟疾病reversed gene expression相关性越大，跟化合物XXX的基因表达相关性越小越好，综合考虑的话，前者除以后者的商越大，组合的后效果好的可能性越大。

In [ ]:
df_merged_cor = df_merged.corr()

In [ ]:
df_merged_cor

In [ ]:
ax = sns.clustermap(df_merged_cor, standard_scale=1, cmap="vlag")
plt.show()

In [ ]:
df_merged_cor_sorted = df_merged_cor.sort_values(by=['breast_cancer_reverse_gene_expression'], ascending=False)
df_merged_cor_sorted_no_disease_row = df_merged_cor_sorted.drop(index='breast_cancer_reverse_gene_expression')
df_cor_1 = df_merged_cor_sorted_no_disease_row.drop('breast_cancer_reverse_gene_expression', 1)
df_cor_2 = df_merged_cor_sorted_no_disease_row['breast_cancer_reverse_gene_expression']
df_cor_1_transpose = df_cor_1.transpose()
df_cor_final = pd.merge(df_cor_1_transpose, df_cor_2, left_index=True, right_index=True)
df_cor_final = df_cor_final.sort_values(by=['breast_cancer_reverse_gene_expression'], ascending=False)
df_cor_final

In [ ]:
col_names = df_cor_final.columns
for sample in col_names:
    if sample != "breast_cancer_reverse_gene_expression":  
        df_cor_final[sample+'_rank_score'] = round(df_cor_final['breast_cancer_reverse_gene_expression']/df_cor_final[sample], 2)
        plot = sns.scatterplot(data=df_cor_final, x=sample, y="breast_cancer_reverse_gene_expression")
        plot.set_xlabel(("correlation with " + sample), fontsize = 20)
        plot.set_ylabel("correlation with disease", fontsize = 20)
        plt.show()

In [ ]:
df_cor_final.to_csv('/Users/yanlixu/Desktop/pingan/合作/CDK12/220608/df_cor_final.csv')

In [ ]:
df_cor_final.to_csv('df_cor_final.csv')